# Traitement des xml pour leur mise en forme avant import dans la base psql

In [7]:
import glob, os
import xml.etree.ElementTree as etree
import pandas as pd
import csv
import requests
import datetime
import time

from sqlalchemy import create_engine
pd.set_option('display.max_columns', 100)

In [8]:
def remplacer(node):
    try : 
        return node.text
    except : 
        return ''

In [9]:
def traiterContrat (contrat, typeContrat, nomFichierSource, numContrat) :
    # Cas où l'id a été oublié
    try : 
        idContrat = contrat.find('id')
        idContrat = idContrat.text
    except : # Création d'un id unique --> a retravailler
        idContrat = nomFichierSource + '_' + str(numContrat)
        
    # Type de contrat
    if typeContrat == "marche" :
        try :
            uidContrat = idContrat + '_' + contrat.find('acheteur/id').text
        except : 
            uidContrat = idContrat + '_' + 'acheteurManquant'
    if typeContrat == "contrat-concession" :
        try : 
            uidContrat = idContrat + '_' +  contrat.find('autoriteConcedante/id').text
        except : 
            idContrat = idContrat + '_' + 'autoriteConcedanteManquante'
    
    #### Infos générales relatives au marché
    infosG = {'idContrat' : idContrat, 'uidContrat' : uidContrat, 'typeContrat' : typeContrat}
    infosG['nature'] = remplacer(contrat.find('nature'))
    infosG['objet'] = remplacer(contrat.find('objet'))
    infosG['codeCPV'] = remplacer(contrat.find('codeCPV'))
    infosG['procedure'] = remplacer(contrat.find('procedure'))
    infosG['lieuExecTypeCode'] = remplacer(contrat.find('lieuExecution/typeCode'))
    infosG['lieuExecCode'] = remplacer(contrat.find('lieuExecution/code'))
    infosG['lieuExecNom'] = remplacer(contrat.find('lieuExecution/nom'))
    infosG['dureeMois'] = remplacer(contrat.find('dureeMois'))
    infosG['dateNotification'] = remplacer(contrat.find('dateNotification'))[:10]
    infosG['datePublicationDonnees'] = remplacer(contrat.find('datePublicationDonnees'))[:10]
    infosG['montant'] = remplacer(contrat.find('montant'))
    infosG['formePrix'] = remplacer(contrat.find('formePrix'))
    
    # Partie spécifique aux concessions
    infosG['dateSignature'] = remplacer(contrat.find('dateSignature'))[:10]
    infosG['dateDebutExecution'] = remplacer(contrat.find('dateDebutExecution'))[:10]
    infosG['valeurGlobale'] = remplacer(contrat.find('valeurGlobale'))
    infosG['montantSubventionPublique'] = remplacer(contrat.find('montantSubventionPublique'))
    
    #### Infos relatives à l'acheteur / autorité concédante
    infosA = {'idContrat' : idContrat, 'uidContrat' : uidContrat}
    if typeContrat == "marche" :
        infosA['idAcheteur'] = remplacer(contrat.find('acheteur/id'))
        infosA['nomAcheteur'] = remplacer(contrat.find('acheteur/nom'))
    if typeContrat == "contrat-concession" :
        infosA['idAcheteur'] = remplacer(contrat.find('autoriteConcedante/id'))
        infosA['nomAcheteur'] = remplacer(contrat.find('autoriteConcedante/nom'))
        
    #### Infos relatives aux titulaires / Concessionnaires / En attendant de le faire en une fois
    infosT = []
    for titulaire in contrat.findall('titulaires/titulaire') :
        infosTitulaire = {'idContrat' : idContrat, 'uidContrat' : uidContrat}
        infosTitulaire['typeIdentifiant'] = remplacer(titulaire.find('typeIdentifiant'))
        infosTitulaire['idTitulaire'] = remplacer(titulaire.find('id'))
        infosTitulaire['denominationSocialeTitulaire'] = remplacer(titulaire.find('denominationSociale'))
        infosT.append(infosTitulaire)
    for titulaire in contrat.findall('concessionnaires/concessionnaire') : 
        infosTitulaire = {'idContrat' : idContrat, 'uidContrat' : uidContrat}
        infosTitulaire['typeIdentifiant'] = remplacer(titulaire.find('typeIdentifiant'))
        infosTitulaire['idTitulaire'] = remplacer(titulaire.find('id'))
        infosTitulaire['denominationSocialeTitulaire'] = remplacer(titulaire.find('denominationSociale'))
        infosT.append(infosTitulaire)
                    
    return({'infosG' : infosG, 'infosA' : infosA, 'infosT' : infosT})

## Boucle sur les fichiers pour extraire les informations

In [10]:
infosG_new = []
infosA_new = []
infosT_new = []

In [11]:
for file in glob.glob("data/*.xml") :
    print("Traitement de : " + file)
    try : 
        tree = etree.parse(file)
        root = tree.getroot()
    except : 
        print("Problème avec le fichier : " + file)
        continue
        
    id_file = file
    c = 0
    for contrat in root : 
        c = c+1
        informations = traiterContrat(contrat, contrat.tag, id_file, c)
        #print(informations)
        infosG_new.append(informations['infosG'])
        infosA_new.append(informations['infosA'])
        for element in informations['infosT'] : # Boucle car plusieurs titulaires possibles
            infosT_new.append(element)

Traitement de : data\decp.xml


In [12]:
# Passage en tableau
infosG_new = pd.DataFrame(infosG_new)
infosA_new = pd.DataFrame(infosA_new)
infosT_new = pd.DataFrame(infosT_new)

### Nettoyage des données 

**Nature de marché**

In [13]:
infosG_new.loc[infosG_new['nature'] == 'CONCESSION DE TRAVAUX',  'nature'] = 'Concession de travaux'
infosG_new.loc[infosG_new['nature'] == 'CONCESSION DE SERVICE',  'nature'] = 'Concession de service'
infosG_new.loc[infosG_new['nature'] == 'CONCESSION DE SERVICE PUBLIC',  'nature'] = 'Concession de service public'
infosG_new.loc[infosG_new['nature'] == 'DELEGATION DE SERVICE PUBLIC',  'nature'] = 'Délégation de service public'
infosG_new.loc[infosG_new['nature'] == 'MARCHE',  'nature'] = 'Marché'
infosG_new.loc[infosG_new['nature'] == 'MARCHE DE PARTENARIAT',  'nature'] = 'Marché de partenariat'
infosG_new.loc[infosG_new['nature'] == 'ACCORD-CADRE',  'nature'] = 'Accord-cadre'
infosG_new.loc[infosG_new['nature'] == 'MARCHE SUBSEQUENT',  'nature'] = 'Marché subséquent'

**Code CPV**

In [14]:
cpv = pd.read_csv('donneesComplementaires/genealogie_cpv.csv', sep = ';', encoding='latin-1')
cpv = cpv[['CODE', 'FR']]
cpv = cpv.rename(columns={'CODE': 'codeCPV', 'FR' : 'nomCPV'})
cpv['codeCPV'] = cpv['codeCPV'].str.slice(0,8)
cpv.head()

,codeCPV,nomCPV
0,03000000,"Produits agricoles, de l'élevage, de la pêche,..."
1,03100000,Produits agricoles et produits de l'horticulture
2,03110000,"Produits agricoles, produits de la culture mar..."
3,03111000,Graines
4,03111100,Graines de soja


In [15]:
infosG_new['codeCPV'] = infosG_new['codeCPV'].str.slice(0,8)
infosG_new = infosG_new.merge(cpv, on='codeCPV', how = "left")

**Lieux d'exécution**

In [16]:
infosG_new['lieuExecCodeDep'] = infosG_new['lieuExecCode']
infosG_new['lieuExecCodeDep'] = infosG_new['lieuExecCodeDep'].str.replace("FR([0-9]{5})", r'\1')

# Des erreurs
# Cas des communes qui sont renseignées en nom au lieu de code
infoMauvaiseCase = infosG_new['lieuExecNom'].str.contains('[0-9]{5}') &  (infosG_new['lieuExecCode'].str.contains('[0-9]{5}') == False)
infosG_new.loc[infoMauvaiseCase, 'lieuExecCodeDep'] = infosG_new['lieuExecNom']

# Cas des gens qui n'ont pas du tout compris comment remplir
loireAtlantique = infosG_new['lieuExecNom'].str.contains('TOUTES LES COMMUNES DE LOIRE-ATLANTIQUE')
infosG_new.loc[loireAtlantique, 'lieuExecCodeDep'] = '44'
infosG_new.loc[loireAtlantique, 'lieuExecTypeCode'] = 'Code département'

pyrAtlantique = infosG_new['lieuExecNom'].str.contains('Département des Pyrénées-Atlantiques')
infosG_new.loc[pyrAtlantique, 'lieuExecCodeDep'] = '64'
infosG_new.loc[pyrAtlantique, 'lieuExecTypeCode'] = 'Code département'

# Multiples erreurs sans nom
infosG_new['lieuExecCodeDep'] = infosG_new['lieuExecCodeDep'].str.replace("^0([0-9]{2})$", r'\1')

# Traitement du résidu
infosG_new.loc[infosG_new['lieuExecCode'] == 'FRK26',  'lieuExecCodeDep'] = '21-58-71-89'
infosG_new.loc[infosG_new['lieuExecNom'] == 'Champagne ardenne',  'lieuExecCodeDep'] = '08-10-51-52'
infosG_new.loc[infosG_new['lieuExecCode'] == 'FRY40',  'lieuExecCodeDep'] = '974'
infosG_new.loc[infosG_new['lieuExecCode'] == 'FR',  'lieuExecCodeDep'] = 'FRANCE'

# Cas de la Corse
corse2A = [v in ['201', '202'] for v in infosG_new['lieuExecCodeDep'].str.slice(0,3)]
corse2B = [v in ['203', '204', '205', '206', '207'] for v in infosG_new['lieuExecCodeDep'].str.slice(0,3)]
infosG_new.loc[corse2A, 'lieuExecCodeDep'] = '2A'
infosG_new.loc[corse2B, 'lieuExecCodeDep'] = '2B'

In [17]:
## Tout ce qui contient plus de 3 caractères (canton, arrondissement, commune)
# Cas des dom
dom = infosG_new['lieuExecCode'].str.contains('^97[0-9]{3}')
infosG_new.loc[dom, 'lieuExecCodeDep'] = infosG_new['lieuExecCodeDep'].str.replace("97([0-9]{1})([0-9]*)", r'97\1')

metro = infosG_new['lieuExecCode'].str.contains('^97[0-9]{3}') == False
infosG_new.loc[metro,'lieuExecCodeDep'] = infosG_new['lieuExecCodeDep'].str.replace("^([0-9]{2})([0-9]*)$", r'\1')

**Montant** -- retrait des virgules

In [18]:
infosG_new['montant'] = infosG_new['montant'].str.replace("^([0-9]*)(\.[0-9]{2})([0-9]*)$", r'\1')

In [19]:
# Réagencement des colonnes
infosG_new = infosG_new[['idContrat', 'uidContrat', 'typeContrat', 'nature', 'objet', 
                         'codeCPV', 'nomCPV', 'procedure', 
                         'lieuExecTypeCode', 'lieuExecCode', 'lieuExecNom', 'lieuExecCodeDep',
                         'dureeMois', 'dateNotification', 'datePublicationDonnees', 
                         'montant', 'formePrix', 'dateSignature', 'dateDebutExecution', 
                         'valeurGlobale', 'montantSubventionPublique']]
infosA_new = infosA_new[['idContrat', 'uidContrat', 'idAcheteur', 'nomAcheteur']]
infosT_new = infosT_new[['idContrat', 'uidContrat', 'typeIdentifiant', 'idTitulaire', 
                         'denominationSocialeTitulaire']]

### Travail sur SIRENE pour homogénéiser les noms des acheteurs et titulaires

In [20]:
config = pd.read_csv('config.csv', header=None)
id = config[0][0]
pwd = config[0][1]
host = config[0][2]
engine = create_engine('postgresql://%s:%s@%s/decp'%(id, pwd, host))

In [21]:
def infosSirene(listeSiren):
    listeSiren = ["'" + x + "'" for x in listeSiren if str(x) != 'nan']
    listeFormatee = ",".join([str(siren) for siren in listeSiren])
    infos = pd.read_sql("SELECT * FROM sirene WHERE siren IN ({ls})".format(ls = listeFormatee), 
                        engine)
    return(infos)

### Acheteurs

In [22]:
infosA_new['sirenAcheteur'] = infosA_new['idAcheteur'].str.slice(0, 9)
acheteurSiren = infosSirene(infosA_new['sirenAcheteur'].unique())

In [23]:
acheteurSiren = acheteurSiren[['siren', 'denominationunique']]
infosA_new = infosA_new.merge(acheteurSiren, left_on='sirenAcheteur', right_on='siren', how = "left")
infosA_new['denominationunique'].fillna(infosA_new['nomAcheteur'], inplace=True)
infosA_new = infosA_new.rename(columns={'denominationunique': 'nomSirenAcheteur'})
del infosA_new['siren']

### Titulaires

In [24]:
infosT_new['sirenTitulaire'] = infosT_new['idTitulaire'].str.slice(0,9)
titulairesSiren = infosSirene(infosT_new['sirenTitulaire'].unique())

In [25]:
titulairesSiren = titulairesSiren[['siren', 'denominationunique']]
titulairesSiren['denominationunique'] = titulairesSiren['denominationunique'].str.replace("(.*)\((.*)\)", r'\1')
infosT_new = infosT_new.merge(titulairesSiren, left_on='sirenTitulaire', right_on='siren', how = "left")

In [26]:
infosT_new['denominationunique'].fillna(infosT_new['denominationSocialeTitulaire'], inplace=True)
infosT_new = infosT_new.rename(columns={'denominationunique': 'nomSirenTitulaire'})
del infosT_new['siren']

In [29]:
infosG_new.to_csv('Contrats.csv', index=False, sep= '|', quoting=csv.QUOTE_NONNUMERIC, encoding='utf-8')
infosA_new.to_csv('Acheteurs.csv', index=False, sep= '|', quoting=csv.QUOTE_NONNUMERIC, encoding='utf-8')
infosT_new.to_csv('Titulaires.csv', index=False, sep= '|', quoting=csv.QUOTE_NONNUMERIC, encoding='utf-8')